In [1]:
# GOAL: convert whole paragraph from biblegateway url to colored paragraph in Greek, 
#       depending on the length of parsing text
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re

# load Greek bible book from biblegateway.com
url_g = input("Enter url from biblegateway in SBLGNT:")
# Example: https://www.biblegateway.com/passage/?search=Luke+1&version=SBLGNT
req_g = requests.get(url_g)
soup_g = BeautifulSoup(req_g.content, 'html.parser')

# initial clean 
paragraph_g = []
for i in soup_g.find_all("p"):
    para_g = i.get_text()
    para_g = para_g.replace("\xa0","")
    paragraph_g.append(para_g)
paragraph_g = ''. join(paragraph_g)
paragraph_g = paragraph_g.split()

# remove "()","[]" in the text
# getting rid of alphabet in "(A)","[a]" 
for i in range(len(paragraph_g)):
    paragraph_g[i] = re.sub("\(.*?\)","",paragraph_g[i])
for i in range(len(paragraph_g)):
    paragraph_g[i] = re.sub("\[.*?\]","",paragraph_g[i])

# separate numeric from text
a_g = []
for i in range(len(paragraph_g)):
    parts_g = re.findall('\d+|\D+', paragraph_g[i])
    a_g.append(parts_g)
paragraph_g = [item for parts_g in a_g for item in parts_g]

# get rid of unwanted texts at the bottom
for i in range(len(paragraph_g)):
    if "quotations" in paragraph_g[i]:
        break
del paragraph_g[i:len(paragraph_g)]

for j in reversed(range(len(paragraph_g))):
    if "." in paragraph_g[j]:
        period_index_g = paragraph_g[j].index(".")
        paragraph_g[j] = paragraph_g[j][:period_index_g+1]
        break
del paragraph_g[j+1:len(paragraph_g)]
paragraph_g[0] = '1'

#list of numeric
num_g = ""
num_lst_g = []
for i in range(len(paragraph_g)):
    if (paragraph_g[i].isdigit()):
        num_g = paragraph_g[i]
        num_lst_g.append(num_g)

# list of index of each numeric, and the last index of the text
def get_index(paragraph,num_lst):  
    index_list = [] 
    for i in range(len(num_lst)):
        if num_lst[i] in paragraph:
            index = paragraph.index(str(num_lst[i]))
            index_list.append(index)
    last = len(paragraph)
    index_list.append(last)
    return index_list
index_list = get_index(paragraph_g, num_lst_g)

def get_index_only(paragraph,num_lst):  
    index_list_only = []
    for i in range(len(num_lst)):
        if num_lst[i] in paragraph:
            index = paragraph.index(str(num_lst[i]))
            index_list_only.append(index)
    return index_list_only

# list of each verse
def word_to_sentence(paragraph,index_list): 
    verse = []
    paragraph_Greek = []
    for i in range(len(index_list)-1):
        start = index_list[i]
        end = index_list[i+1]
        verse.append(paragraph[start:end])
        verse = [' '.join(j) for j in verse]
        paragraph_Greek.append(verse)
        verse = []
    return paragraph_Greek
paragraph_Greek = word_to_sentence(paragraph_g,index_list)

Enter url from biblegateway in SBLGNT:https://www.biblegateway.com/passage/?search=Luke+2&version=SBLGNT


In [2]:
from itertools import zip_longest

# load bible book from blueletterbible.org
parsing_text = input("Enter txt file name:")
# Example: Greek luke1:7-8.txt
with open(parsing_text, 'r') as f:
    parsing = f.readlines()

# arrange text into group of 3: (English, Greek, Parsing Code)
def grouper(iterable, n, fillvalue=None):
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)
parsing = list(grouper(parsing,3,'-'))

# get rid of \n
for i in range(len(parsing)):
    parsing[i] = list(parsing[i])
    for j in range(len(parsing[i])):
        parsing[i][j] = parsing[i][j].replace("\n","")
    
# group text by parsing code
code = []
for a in parsing:
    if a[2] not in code:
        code.append(a[2])
        code.append([])
for b in code:
    for a in parsing:
        if b == a[2]:
            code[code.index(b)+1].append([a[0], a[1]])
# for no in range(0,len(code),2):
#     print([code[no]] + [a for a in code[no+1]])

# apply color   
RESET = '\033[0m'
def get_color_escape(r, g, b, background=True):
    return '\033[{};2;{};{};{}m'.format(48 if background else 38, r, g, b)

COLOR = {
    "NOM": get_color_escape(0, 255, 0),
    "ACC": get_color_escape(255, 0, 0),
    "DAT": get_color_escape(128, 128, 128),
    "GEN": get_color_escape(0, 128, 128),
    "PREP": get_color_escape(128, 128, 0),
    "VERB": get_color_escape(255, 255, 0),
    "PART": get_color_escape(0, 255, 255),
    "INFI": get_color_escape(255, 0, 255),
    "CONJ": "\033[4m",
    "BLACK": "\033[0m"
}

# define function to convert parsing code to color code
def mini_code_to_color(code):  
    if "D" in code:
        return COLOR["DAT"]
    elif "G" in code:
        return COLOR["GEN"]
    elif "V" in code:
        return COLOR["VERB"]
    if i+2 < len(code):
        if "N" in code[i+2]:
            return COLOR["NOM"]
        elif "A" in code[i+2]:
            return COLOR["ACC"]  
        else:
            return COLOR["BLACK"]
    else:
        return COLOR["BLACK"]

def code_to_color(code):  
    if "PREP" in code:
        return COLOR["PREP"]
    if "CONJ" in code:
        return COLOR["CONJ"]
    for i in range(len(code)):
        if "D" in code:
            return COLOR["DAT"]
        if "G" in code:
            return COLOR["GEN"]
        if "V" in code:
            if "PAN" in code:
                return COLOR["INFI"]
            elif "PMN" in code:
                return COLOR["INFI"]
            elif "PPN" in code:
                return COLOR["INFI"]
            elif "AAN" in code:
                return COLOR["INFI"]
            elif "AMN" in code:
                return COLOR["INFI"]
            elif "APN" in code:
                return COLOR["INFI"]
            elif "P-" in code[i+4:i+6]:
                return COLOR["PART"]
            else:
                return COLOR["VERB"]
        if "N" in code[i]:
            if i+2 < len(code):
                if "N" in code[i+2]:
                    return COLOR["NOM"]
                if "A" in code[i+2]:
                    return COLOR["ACC"]
                else:
                    mini_code_to_color(code[i+2])
            else:
                return COLOR["NOM"]
        elif "A" in code[i]:
            if i+2 < len(code):
                if "A" in code[i+2]:
                    return COLOR["ACC"]
                if "N" in code[i+2]:
                    return COLOR["NOM"]
                else:
                    mini_code_to_color(code[i+2])
            else:
                return COLOR["ACC"]
        elif "T" in code[i]:
            if i+2 < len(code):
                if "A" in code[i+2]:
                    return COLOR["ACC"]
                if "N" in code[i+2]:
                    return COLOR["NOM"]
                else:
                    mini_code_to_color(code[i+2])
            else:
                return "BLACK"
        elif "P" in code[i]:
            if i+2 < len(code):
                if "A" in code[i+2]:
                    return COLOR["ACC"]
                if "N" in code[i+2]:
                    return COLOR["NOM"]
                else:
                    mini_code_to_color(code[i+2])
            else:
                return "BLACK"
    else:
        return COLOR["BLACK"]

# colored text in English(reversed)
def reversed_English(parsing):
    colored_English = ""
    for i in range(len(parsing)):
        colored_English += code_to_color(parsing[i][-1])+parsing[i][0]+RESET + " "
    return colored_English

# colored text in Greek(forward)
def forward_Greek(parsing):
    colored_Greek = ""
    for i in range(len(parsing)):
        colored_Greek += code_to_color(parsing[i][-1])+parsing[i][1]+RESET + " " 
    return colored_Greek_reversed

# let user enter starting verse number and ending verse number
number_start = int(input("Enter start verse:"))
number_end = int(input("Enter end verse:"))
user_start = number_start
user_end = number_end

Enter txt file name:Luke2:16-21_Greek.txt
Enter start verse:16
Enter end verse:21


In [3]:
# get token from txt file
def token_Greek(parsing):
    parsing_Greek = []
    for i in range(len(parsing)):
        parsing_Greek.append(parsing[i][1])
    return parsing_Greek

# arrange TE_combine so that it fits regex
TE = token_Greek(parsing)

index_list_only = get_index_only(paragraph_g,num_lst_g)
start = index_list_only[user_start-1]
end = index_list_only[user_end]
user_paragraph = paragraph_g[start:end]
user_index_list = get_index(user_paragraph,num_lst_g)
user_paragraph = word_to_sentence(user_paragraph,user_index_list)

def flatten(t):
    return [item for sublist in t for item in sublist]

def sep_punctuation(user_paragraph):
    itr_sep_punctuation = []
    lst_sep_punctuation = []
    for i in range(len(user_paragraph)):
        find_regex = r'\w+|[.,!?:;]'
        itr_sep_punctuation = re.findall(find_regex, user_paragraph[i][0])
        lst_sep_punctuation.append(itr_sep_punctuation)
    return lst_sep_punctuation
sep = sep_punctuation(user_paragraph)

In [4]:
# get number index for user_t
# print complete color-coded user paragraph with line break
import string
user_t = flatten(sep)
user_t_index = get_index(user_t,num_lst_g)
count = 0
i = 0
j = 0
while j < len(parsing):
    while i < len(user_t):
        if user_t[i].isdigit():
            j = j
            user_t[i] = user_t[i]
        elif user_t[i] in string.punctuation:
            j = j
            user_t[i] = user_t[i]
        else:
            user_t[i] = code_to_color(parsing[j][-1])+parsing[j][1]+RESET
            j += 1
        i += 1
    j += 1
            
final_user = word_to_sentence(user_t,user_t_index)
for i in range(len(final_user)):
    print(final_user[i][0])

16 καὶ ἦλθαν σπεύσαντες καὶ ἀνεῦραν τήν τε Μαριὰμ καὶ τὸν Ἰωσὴφ καὶ τὸ βρέφος κείμενον ἐν τῇ φάτνῃ
17 ἰδόντες δὲ ἐγνώρισαν περὶ τοῦ ῥήματος τοῦ λαληθέντος αὐτοῖς περὶ τοῦ παιδίου τούτου .
18 καὶ πάντες οἱ ἀκούσαντες ἐθαύμασαν περὶ τῶν λαληθέντων ὑπὸ τῶν ποιμένων πρὸς αὐτούς ,
19 ἡ δὲ Μαριὰμ πάντα συνετήρει τὰ ῥήματα ταῦτα συμβάλλουσα ἐν τῇ καρδίᾳ αὐτῆς .
20 καὶ ὑπέστρεψαν οἱ ποιμένες δοξάζοντες καὶ αἰνοῦντες τὸν θεὸν ἐπὶ πᾶσιν οἷς ἤκουσαν καὶ εἶδον καθὼς ἐλαλήθη πρὸς αὐτούς .
21 καὶ ὅτε ἐπλήσθησαν ἡμέραι ὀκτὼ τοῦ περιτεμεῖν αὐτόν , καὶ ἐκλήθη τὸ ὄνομα αὐτοῦ Ἰησοῦς , τὸ κληθὲν ὑπὸ τοῦ ἀγγέλου πρὸ τοῦ συλλημφθῆναι αὐτὸν ἐν τῇ κοιλίᾳ .
